In [ ]:
!pip install requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import re
from datetime import datetime


def scrape(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        main_wrap_div = soup.find("div", class_="col-12 col-sm-12 col-md-8 col-lg-6 col-xl-12 mainContentWrapNewsFeed")

        if main_wrap_div:
            sub_div_blocks = main_wrap_div.find_all("div", class_="listnewsContAdj col ps-0 live-listing")

            for sub_div in sub_div_blocks:
                link = sub_div.find("a")
                if link and 'href' in link.attrs:
                    article_url = link['href']
                    # print(article_url)
                    article_response = requests.get(article_url, headers=headers)

                    if article_response.status_code == 200:
                        article_soup = BeautifulSoup(article_response.content, "html.parser")

                        h2_story_summary = article_soup.find('h2', class_='story-summary')
                        if h2_story_summary:
                            story_summary_content = h2_story_summary.text.strip()
                            # print(story_summary_content)
                        else:
                            continue

                        if "https://" in story_summary_content:
                          story_summary_content=story_summary_content.replace("https://","")

                        if ":" in story_summary_content:
                          claim = story_summary_content.split(':')[1]
                          if not claim:
                            continue
                          if claim[0]==" ":
                            claim=claim[1:]
                        elif "|" in story_summary_content:
                          claim = story_summary_content.split('|')[1]
                          if not claim:
                            continue
                          if claim[0]==" ":
                            claim=claim[1:]
                        else:
                          claim=story_summary_content


                        # if ").." in claim:
                        #   claim = re.sub(r'\([^)]*\)..', '', claim)
                        # elif ")," in claim:
                        #   claim = re.sub(r'\([^)]*\),', '', claim)
                        # elif ")." in claim:
                        #   claim = re.sub(r'\([^)]*\).', '', claim)
                        # elif ") " in claim:
                        #   claim = re.sub(r'\([^)]*\) ', '', claim)
                        # else:
                        #   claim = re.sub(r'\([^)]*\)', '', claim)

                        # non_english_text=[]
                        # for text in claim:
                        #   if "(" in text:
                        #     text = text.replace("(", "")
                        #   if ")" in text:
                        #     text = text.replace(")", "")
                        #   if not text.isalpha():
                        #     non_english_text.append(text)

                        # claim = ' '.join(claim).strip()
                        # print(claim)

                        first_news_discription_div = article_soup.find('div', class_='newsDiscription')

                        if first_news_discription_div:
                            first_p_within_news_discription = first_news_discription_div.find('p')
                            if first_p_within_news_discription:
                                news_discription_content = first_p_within_news_discription.text.strip()
                            else:
                                continue
                        else:
                            continue

                        if "https://" in news_discription_content:
                          news_discription_content=news_discription_content.replace("https://","")

                        if ":" in news_discription_content:
                          fact = news_discription_content.split(':')[1]
                          if not fact:
                            continue
                          if fact[0]==" ":
                            fact=fact[1:]
                        elif "|" in news_discription_content:
                          fact = news_discription_content.split('|')[1]
                          if not fact:
                            continue
                          if fact[0]==" ":
                            fact=fact[1:]
                        else:
                          fact=news_discription_content
                        # print(fact)

                        # if ").." in fact:
                        #   fact = re.sub(r'\([^)]*\)..', '', fact)
                        # elif ")," in fact:
                        #   fact = re.sub(r'\([^)]*\),', '', fact)
                        # elif ")." in fact:
                        #   fact = re.sub(r'\([^)]*\).', '', fact)
                        # elif ") " in fact:
                        #   fact = re.sub(r'\([^)]*\) ', '', fact)
                        # else:
                        #   fact = re.sub(r'\([^)]*\)', '', fact)

                        # non_english=[]
                        # for text in fact:
                        #   if "(" in text:
                        #     text = text.replace("(", "")
                        #   if ")" in text:
                        #     text = text.replace(")", "")
                        #   if not text.isalpha():
                        #     non_english.append(text)

                        # fact = ' '.join(fact).strip()
                        # print(fact)

                        fact_check="Not Fake"

                        author_name= article_soup.find('span', class_='authorName pb-0')
                        if author_name:
                            author_name = author_name.find('a')
                            if author_name:
                                author= author_name.text.strip()
                                # print(author)
                            else:
                              continue
                        else:
                            continue

                        date_time_div = article_soup.find('div', class_='dateTimess leftDots')
                        # print(date_time_div)

                        if date_time_div:
                            date_time = date_time_div.text.strip()
                            date_time = date_time.rsplit('  ', 1)[0]
                            date_time = datetime.strptime(date_time, '%b %d, %Y %I:%M %p')
                            date = date_time.strftime('%Y-%m-%d')
                            time = date_time.strftime('%H:%M:%S')
                        else:
                            continue
                            # print(date_time,type(date_time))
                        # print(date_time)

                        # print(date)
                        # print(time)

                        # Prepare the data for CSV
                        data = {
                            'Claim': claim,
                            'Fact': fact,
                            'Fact-check': fact_check,
                            'Reviewer': author,
                            'Date(YYYY-MM-DD)': date,
                            'Time' : time
                        }

                        # Write to CSV
                        with open('output.csv', mode='a', newline='', encoding='utf-8') as file:
                            writer = csv.DictWriter(file, fieldnames=data.keys())
                            file.seek(0, 2)  # Move to the end of the file
                            if file.tell() == 0:
                                writer.writeheader()
                            writer.writerow(data)

                        print(f'Data from {article_url} written to output.csv.')

                    else:
                        print(f"Failed to retrieve the article at {article_url}. Status code: {article_response.status_code}")
                else:
                    print("No valid link found in the sub div.")
        else:
            print("No valid sub div blocks found.")
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


# url = "https://telugu.hindustantimes.com/national-international/page-52"
# scrape(url,52)

for i in range(176, 300):
    print(i)
    url = f"https://telugu.hindustantimes.com/national-international/page-{i}"
    scrape(url)

print("")
print("completed")


176
Data from https://telugu.hindustantimes.com/national-international/india-will-continue-to-explore-ways-to-bring-back-kohinoor-from-uk-centre-121665800155518.html written to output.csv.
Data from https://telugu.hindustantimes.com/national-international/nearly-100-students-fall-sick-after-gas-leak-at-tamil-nadu-school-121665798800819.html written to output.csv.
Data from https://telugu.hindustantimes.com/national-international/rajnath-singh-launches-website-for-citizens-to-contribute-to-martyrs-families-121665766610444.html written to output.csv.
Data from https://telugu.hindustantimes.com/national-international/nuclearpowered-submarine-ins-arihant-carries-successful-launch-of-ballistic-missile-121665765015283.html written to output.csv.
Data from https://telugu.hindustantimes.com/national-international/decomposed-body-of-iit-kharagpur-student-recovered-from-hostel-room-121665762775882.html written to output.csv.
Data from https://telugu.hindustantimes.com/national-international/no-g